In [ ]:
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

import time
import re
import os

browser = webdriver.Firefox()

<br>

---
* <u>Wait for browser to load<u/>

In [ ]:
word_list = '''Parler to speak
    Être to be
    Avoir to have
    Aller to go
    Faire to make
    Prendre to take
    Venir to come
    Pouvoir to able
    Vouloir to want
    Mettre to put'''

In [ ]:
word_parse = re.split(r"\s.o\s|\n\s*|\n", word_list.lower())

conj_list = []
for k in range(len(word_parse)):
    if not k%2:
        conj_list.append([word_parse[k].lower(), word_parse[k+1].lower()])
conj_list

In [ ]:
starttime = time.time()


french_page = "https://www.the-conjugation.com/french/"
english_page = "https://www.the-conjugation.com/"
temps_de_verbe = ["Indicatif", "Passé simple"]
verb_tenses = ["Simple present", "Simple past"]
output = ""

for pair in conj_list:
     # ****** French ******
    browser.get(french_page)

    timeout = 10
    try:
        element_present = expected_conditions.presence_of_element_located((By.TAG_NAME, 'h1'))
        WebDriverWait(browser, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
    finally:
        print("Page loaded")

    search_bar = browser.find_element(By.NAME, "q")
    webdriver.ActionChains(browser).click(search_bar).perform()
    search_bar.send_keys(pair[0] + Keys.RETURN)

    time.sleep(3)
    html_français = browser.page_source
    
    # ****** English ******
    browser.get(english_page)

    timeout = 10
    try:
        element_present = expected_conditions.presence_of_element_located((By.TAG_NAME, 'h1'))
        WebDriverWait(browser, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
    finally:
        print("Page loaded")

    search_bar = browser.find_element(By.NAME, "q")
    webdriver.ActionChains(browser).click(search_bar).perform()
    search_bar.send_keys(pair[1] + Keys.RETURN)
    
    time.sleep(3)
    english_html = browser.page_source



    # Parse source code and format for upload to flashcard website.
    for i in range(len(verb_tenses)):
        try:
            temp = re.findall(fr'{temps_de_verbe[i]}.*?tempscorps.>(.+?)div', html_français) # list of a single string.
            conj_present = re.split(r'</b><br>|<br>', temp[0]) # list of raw words.
        except IndexError:
            print(f"IndexError: {pair[0]} failed.")
            continue
        try:
            tense = re.findall(fr'{verb_tenses[i]}.*?tempscorps.>(.+?)div', english_html) # list of a single string.
            present_conj = re.split(r'</b><br>|<br>', tense[0]) # list of raw words.
        except IndexError:
            print(f"IndexError: {pair[1]} failed.")
            continue
        # Alternate French and English terms into formatted output.
        for j in range(len(present_conj)):
            if conj_present[j] != "</":
                mot_propre = re.sub("<b>", "", conj_present[j])         
                output = output + mot_propre + ","
            if present_conj[j] != "</":
                clean_word = re.sub("<b>", "", present_conj[j])
                output = output + clean_word + ";"


totaltime = time.time() - starttime
print(f"\n****** {totaltime//60:.0f}m {totaltime%60:.2f}s ******")

In [ ]:
output

<br>

---
* <u>Quit browser when all downloads are done</u>

In [ ]:
browser.quit()